In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
import requests
import pandas as pd

# URL de la primera API para obtener OCID para el año 2021
ocid_search_url = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds?search=agua&year=2021"

# Realiza una solicitud GET para obtener los OCID
response = requests.get(ocid_search_url)

# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    # Convierte la respuesta JSON en un DataFrame de pandas
    data = response.json()
    df_ocids = pd.DataFrame(data['data'])

    # Obtén la lista de OCIDs
    ocid_list = df_ocids['ocid'].tolist()

    # URL de la segunda API para obtener información adicional por OCID
    record_api_url = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/record?ocid="

    # Itera a través de los OCIDs y obtiene información adicional
    all_contract_data = []
    for ocid in ocid_list:
        full_url = record_api_url + ocid
        response = requests.get(full_url)
        if response.status_code == 200:
            contract_data = response.json()
            all_contract_data.append(contract_data)
        else:
            print(f"No se pudo obtener información para OCID: {ocid}")

    # Convierte la lista de resultados en un DataFrame
    df_contract_info = pd.DataFrame(all_contract_data)
    
    # Ahora tienes un DataFrame (df_contract_info) que contiene la información detallada para cada OCID.

    # Puedes manipular y analizar estos datos según tus necesidades.
else:
    print("La solicitud para obtener OCIDs no fue exitosa.")


In [5]:
df = df_contract_info.copy()

In [6]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado df

# Función para explorar y extraer las llaves anidadas en un JSON
def find_nested_keys(data, parent_key='', separator='_'):
    items = {}
    if isinstance(data, list):
        for i, item in enumerate(data):
            items.update(find_nested_keys(item, f"{parent_key}{separator}{i}", separator))
    elif isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{parent_key}{separator}{key}" if parent_key else key
            items.update(find_nested_keys(value, new_key, separator))
    else:
        items[parent_key] = data
    return items

# Aplicar la función a la columna 'releases'
keys_dict = df['releases'].apply(find_nested_keys).apply(pd.Series)

# Concatenar el DataFrame de las llaves extraídas con el DataFrame original
df = pd.concat([df, keys_dict], axis=1)

# Eliminar la columna 'releases' si ya no se necesita
df.drop(columns=['releases'], inplace=True)

# Renombrar las columnas
df.columns = df.columns.str.replace('\d+_', '').str.replace('_', ' ').str.replace('0', '')


/var/folders/s7/2y3h3pk5083g_g_9p2dgfd_c0000gn/T/ipykernel_4354/1514187808.py:20: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  keys_dict = df['releases'].apply(find_nested_keys).apply(pd.Series)


In [7]:
df.head(2)

,uri,license,version,publisher,extensions,publishedDate,publicationPolicy,id,tag,tag 1,tag 2,tag 3,tag 4,bids details id,bids details date,bids details status,bids details tenderers id,bids details tenderers name,bids details 1 id,bids details 1 date,bids details 1 status,bids details 1 tenderers id,bids details 1 tenderers name,bids details 2 id,bids details 2 date,bids details 2 status,bids details 2 tenderers id,bids details 2 tenderers name,bids details 3 id,bids details 3 date,bids details 3 status,bids details 3 tenderers id,bids details 3 tenderers name,bids statistics id,bids statistics date,bids statistics notes,bids statistics value,bids statistics measure,bids statistics 1 id,bids statistics 1 date,bids statistics 1 notes,bids statistics 1 value,bids statistics 1 measure,bids statistics 2 id,bids statistics 2 date,bids statistics 2 notes,bids statistics 2 value,bids statistics 2 measure,bids statistics 3 id,bids statistics 3 date,bids statistics 3 notes,bids statistics 3 value,bids statistics 3 measure,date,ocid,buyer id,buyer name,awards id,awards items id,awards items unit id,awards items unit name,awards items unit value amount,awards items unit value currency,awards items unit scheme,awards items quantity,awards items description,awards items classification id,awards items classification scheme,awards items classification description,awards items additionalClassifications id,awards items additionalClassifications uri,awards items additionalClassifications scheme,awards items additionalClassifications description,awards value amount,awards value currency,awards suppliers id,awards suppliers name,awards description,tender id,tender items id,tender items unit id,tender items unit name,tender items unit value amount,tender items unit value currency,tender items unit scheme,tender items quantity,tender items description,tender items classification id,tender items classification scheme,tender items classification description,tender items additionalClassifications id,tender items additionalClassifications uri,tender items additionalClassifications scheme,tender items additionalClassifications description,tender title,tender value amount,tender value currency,tender status,tender enquiries id,tender enquiries date,tender enquiries answer,tender enquiries author id,tender enquiries author name,tender enquiries description,tender enquiries dateAnswered,tender enquiries 1 id,tender enquiries 1 date,tender enquiries 1 answer,tender enquiries 1 author id,tender enquiries 1 author name,tender enquiries 1 description,tender enquiries 1 dateAnswered,tender enquiries 2 id,tender enquiries 2 date,tender enquiries 2 answer,tender enquiries 2 author id,tender enquiries 2 author name,tender enquiries 2 description,tender enquiries 2 dateAnswered,tender enquiries 3 id,tender enquiries 3 date,tender enquiries 3 answer,tender enquiries 3 author id,tender enquiries 3 author name,tender enquiries 3 description,tender enquiries 3 dateAnswered,tender enquiries 4 id,tender enquiries 4 date,tender enquiries 4 answer,tender enquiries 4 author id,tender enquiries 4 author name,tender enquiries 4 description,tender enquiries 4 dateAnswered,tender enquiries 5 id,tender enquiries 5 date,tender enquiries 5 answer,tender enquiries 5 author id,tender enquiries 5 author name,tender enquiries 5 description,tender enquiries 5 dateAnswered,tender tenderers id,tender tenderers name,tender tenderers 1 id,tender tenderers 1 name,tender awardPeriod endDate,tender awardPeriod maxExtentDate,tender description,tender hasEnquiries,tender tenderPeriod endDate,tender tenderPeriod startDate,tender tenderPeriod maxExtentDate,tender tenderPeriod durationInDays,tender awardCriteria,tender enquiryPeriod endDate,tender enquiryPeriod startDate,tender enquiryPeriod maxExtentDate,tender enquiryPeriod durationInDays,tender procuringEntity id,tender procuringEntity name,tender numberOfTenderers,tender procurementMethod,tender eligibilityCriteria,tender mainProcurementCatego

In [8]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado df

# Función para convertir un texto en estilo CamelCase
def camel_case(text):
    words = text.split()
    # Capitalizar la primera letra de cada palabra excepto la primera
    return words[0] + ''.join(word.capitalize() for word in words[1:])

# Eliminar espacios y saltos de línea en los nombres de las columnas y aplicar estilo CamelCase
df.columns = df.columns.str.replace(r'[\s\n]+', ' ').str.strip().map(camel_case)

# Verificar el nuevo nombre de las columnas
print(df.columns)


Index(['uri', 'license', 'version', 'publisher', 'extensions', 'publishedDate',
       'publicationPolicy', 'id', 'tag', 'tag1',
       ...
       'contractsId', 'contractsValueAmount', 'contractsValueCurrency',
       'contractsPeriodEnddate', 'contractsPeriodStartdate',
       'contractsPeriodDurationindays', 'contractsStatus', 'contractsAwardid',
       'contractsDatesigned', 'partiesAddressPostalcode'],
      dtype='object', length=223)


# Modelo

In [9]:
# Suponiendo que tienes un DataFrame llamado df

# Obtener los nombres de las columnas como una lista
column_names = df.columns.tolist()

# Especifica el nombre del archivo de texto donde deseas exportar los nombres de las columnas
archivo_txt = "nombres_columnas.txt"

# Abre el archivo en modo escritura y escribe los nombres de las columnas
with open(archivo_txt, 'w') as file:
    for columna in column_names:
        file.write(columna + '\n')

# Confirmación de que se ha exportado correctamente
print(f"Los nombres de las columnas se han exportado a {archivo_txt}")


Los nombres de las columnas se han exportado a nombres_columnas.txt


In [10]:
# data=df[["awards items description","tender procurementMethodDetails","parties 1 address locality","awards contractPeriod durationInDays","awards value amount"]]
data=df[["awardsItemsDescription","tenderProcurementmethoddetails","parties1AddressLocality","contractsPeriodDurationindays","awardsValueAmount"]]
# Suponiendo que tienes un DataFrame llamado df

# Define un diccionario de mapeo de nombres de columnas
column_mapping = {
    'awardsItemsDescription': 'description',
    'tenderProcurementmethoddetails': 'procurementMethod',
    'parties1AddressLocality': 'locality',
    'contractsPeriodDurationindays': 'durationInDays',
    'awardsValueAmount': 'estimatedPrice'
}

# Renombra las columnas utilizando el diccionario
data.rename(columns=column_mapping, inplace=True)
data.dropna(inplace=True)
print(data.isnull().sum())
print(data.shape)
data.head(2)

description          0
procurementMethod    0
locality             0
durationInDays       0
estimatedPrice       0
dtype: int64
(5, 5)


/var/folders/s7/2y3h3pk5083g_g_9p2dgfd_c0000gn/T/ipykernel_4354/1443573671.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns=column_mapping, inplace=True)
/var/folders/s7/2y3h3pk5083g_g_9p2dgfd_c0000gn/T/ipykernel_4354/1443573671.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


,description,procurementMethod,locality,durationInDays,estimatedPrice
3,SERVICIOS DE ENSAYO Y ANALISIS DE LAS PROPIEDA...,Contratacion directa,GUAYAQUIL,365.0,12180.0
5,SERVICIOS DE ENSAYO Y ANALISIS DE LAS PROPIEDA...,Contratacion directa,GUAYAQUIL,30.0,4800.0


In [76]:
data.dtypes

description           object
procurementMethod     object
locality              object
durationInDays       float64
estimatedPrice       float64
dtype: object

In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample


# Divide tus datos en características (X) y la variable objetivo (y)
X = data.drop(columns=['estimatedPrice'])
y = data['estimatedPrice']

# Divide tus datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define las transformaciones para diferentes tipos de columnas
numeric_features = ['durationInDays']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = ['procurementMethod', 'locality']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crea el modelo (usaremos RandomForestRegressor como ejemplo)
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Define el pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Entrena el modelo completo
pipeline.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcula el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Configura el número de muestras bootstrap
n_bootstrap_samples = 1000  # Puedes ajustar este valor según tus necesidades

# Crea una lista para almacenar las predicciones bootstrap
bootstrap_predictions = []

# Realiza el bootstrap y almacena las predicciones en la lista
for _ in range(n_bootstrap_samples):
    # Realiza un remuestreo con reemplazo de tus datos de prueba
    X_boot, y_boot = resample(X_test, y_test, random_state=42)
    
    # Realiza predicciones con el modelo en los datos de prueba remuestreados
    y_boot_pred = pipeline.predict(X_boot)
    
    # Agrega las predicciones bootstrap a la lista
    bootstrap_predictions.append(y_boot_pred)

# Calcula los intervalos de confianza del 95%
percentile_2_5 = np.percentile(bootstrap_predictions, 2.5, axis=0)
percentile_97_5 = np.percentile(bootstrap_predictions, 97.5, axis=0)

# Imprime el error cuadrático medio y los intervalos de confianza
print(f'MSE: {mse:.2f}')
print(f'Intervalo de confianza al 95%: ({percentile_2_5[0]:.2f}, {percentile_97_5[0]:.2f})')




MSE: 19599767.80
Intervalo de confianza al 95%: (13572.84, 13572.84)


In [88]:
# Importa la librería necesaria para calcular los IC
from scipy import stats

# Define tus propios datos de entrada
nuevo_description = "SERVICIOS DE ASESORAMIENTO Y REPRESENTACION JURIDICOS EN PROCEDIMIENTOS JUDICIALES RELATIVOS A OTRAS ESFERAS DEL DERECHO, COMO DEFENSA DE UN CASO ANTE LOS TRIBUNALES"
nuevo_procurementMethod = "Asesoría y Patrocinio Jurídico"
nuevo_locality = "BUENA FE"
nuevo_durationInDays = 240  # Tu duración en días aquí

# Crea un DataFrame con tus datos de entrada
nuevos_datos = pd.DataFrame({
    'description': [nuevo_description],
    'procurementMethod': [nuevo_procurementMethod],
    'locality': [nuevo_locality],
    'durationInDays': [nuevo_durationInDays]
})

# Realiza una predicción con tus datos
nueva_prediccion = pipeline.predict(nuevos_datos)

# Calcula el error estándar de las predicciones en el conjunto de prueba
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
std_error = np.sqrt(mse)

# Establece el nivel de confianza al 95%
conf_level = 0.95

# Calcula el valor crítico de Z para el nivel de confianza
z_critical = stats.norm.ppf(1 - (1 - conf_level) / 2)

# Calcula el intervalo de confianza
lower_bound = nueva_prediccion - z_critical * std_error
upper_bound = nueva_prediccion + z_critical * std_error

# Imprime la predicción y el intervalo de confianza
print(f'Predicción de Gasto: ${nueva_prediccion[0]:.2f}')
print(f'Intervalo de confianza al 95%: (${lower_bound[0]:.2f}, ${upper_bound[0]:.2f})')


Predicción de Gasto: $48238.92
Intervalo de confianza al 95%: ($39561.84, $56916.00)


# Modelo 2

In [13]:
# Cargar los datos en un DataFrame de pandas
data = data.copy()


In [14]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors



# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data.drop('estimatedPrice', axis=1), data['estimatedPrice'], test_size=0.2, random_state=42)

# Codificar las variables categóricas
le = LabelEncoder()
for col in X_train.select_dtypes(include='object').columns:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

# Crear una matriz de características TF-IDF para la columna de descripción del producto
vectorizer = TfidfVectorizer()
X_train_desc = vectorizer.fit_transform(X_train['description'])
X_test_desc = vectorizer.transform(X_test['description'])

# Entrenar un modelo de vecinos más cercanos en la matriz de características TF-IDF
nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')
nn.fit(X_train_desc)

# Generar vectores de características para las columnas numéricas y categóricas
X_train_num_cat = X_train.drop('description', axis=1)
X_test_num_cat = X_test.drop('description', axis=1)

# Hacer predicciones para los datos de prueba
y_pred = []
for i in range(len(X_test)):
    desc_vec = X_test_desc[i]
    num_cat_vec = X_test_num_cat.iloc[i].values.reshape(1, -1)
    nn_idx = nn.kneighbors(desc_vec)[1][0][0]
    num_cat_val = X_train_num_cat.iloc[nn_idx].values.reshape(1, -1)
    X_pred = pd.DataFrame(data=np.hstack([desc_vec.toarray(), num_cat_val]), columns=[f'feature_{i}' for i in range(desc_vec.shape[1] + num_cat_val.shape[1])])
    y_pred.append(model.predict(X_pred)[0])

# Evaluar el modelo en los datos de prueba
score = r2_score(y_test, y_pred)
print(f'El coeficiente R^2 del modelo es {score:.2f}')


ValueError: y contains previously unseen labels: 'SERVICIOS DE ENSAYO Y ANALISIS DE LAS PROPIEDADES QUIMICAS Y BIOLOGICAS DEL AGUA'